In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.stats import norm

In [3]:
ticker = "SPY"
data = yf.download(ticker,start="2024-06-01", end="2024-12-01")
print(data)

[*********************100%***********************]  1 of 1 completed

Price        Adj Close       Close        High         Low        Open  \
Ticker             SPY         SPY         SPY         SPY         SPY   
Date                                                                     
2024-06-03  522.734619  527.799988  529.309998  522.599976  529.020020   
2024-06-04  523.318970  528.390015  529.150024  524.960022  526.460022   
2024-06-05  529.538696  534.669983  534.690002  528.729980  530.770020   
2024-06-06  529.528748  534.659973  535.419983  532.679993  534.979980   
2024-06-07  528.885071  534.010010  536.890015  532.539978  533.659973   
...                ...         ...         ...         ...         ...   
2024-11-22  593.512451  595.510010  596.150024  593.150024  593.659973   
2024-11-25  595.525696  597.530029  600.859985  595.200012  599.520020   
2024-11-26  598.635193  600.650024  601.330017  598.070007  598.799988   
2024-11-27  596.821289  598.830017  600.849976  597.280029  600.460022   
2024-11-29  600.528809  602.549988  60

In [4]:
data['Log Returns'] = np.log(data['Close'] / data['Close'].shift(1))
S = data['Close'].iloc[-1]
K = S
T = 0.5
r = 0.05
sigma = data['Log Returns'].std() * np.sqrt(252)

In [6]:
def black_scholes(S, K, T, r, sigma, option_type):
    d1 = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "call":
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Option type must be 'call' or 'put'")
    return price

In [7]:
call_price = black_scholes(S, K, T, r, sigma, option_type="call")
put_price = black_scholes(S, K, T, r, sigma, option_type="put")



In [10]:
print("Call price=",call_price)
print("Put price=",put_price)

Call price= Ticker
SPY    30.665481
Name: 2024-11-29 00:00:00, dtype: float64
Put price= Ticker
SPY    15.788469
Name: 2024-11-29 00:00:00, dtype: float64


In [11]:
def option_greeks(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    delta_call = norm.cdf(d1)
    delta_put = -norm.cdf(-d1)
    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    theta_call = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))
                  - r * K * np.exp(-r * T) * norm.cdf(d2))
    theta_put = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))
                 + r * K * np.exp(-r * T) * norm.cdf(-d2))
    vega = S * norm.pdf(d1) * np.sqrt(T)
    rho_call = K * T * np.exp(-r * T) * norm.cdf(d2)
    rho_put = -K * T * np.exp(-r * T) * norm.cdf(-d2)

    return {
        "delta_call": delta_call, "delta_put": delta_put,
        "gamma": gamma, "theta_call": theta_call, "theta_put": theta_put,
        "vega": vega, "rho_call": rho_call, "rho_put": rho_put
    }

In [12]:
greeks = option_greeks(S, K, T, r, sigma)

In [13]:
print(greeks)

{'delta_call': array([0.62232777]), 'delta_put': array([-0.37767223]), 'gamma': Ticker
SPY    0.006668
Name: 2024-11-29 00:00:00, dtype: float64, 'theta_call': Ticker
SPY   -38.875557
Name: 2024-11-29 00:00:00, dtype: float64, 'theta_put': Ticker
SPY   -9.491908
Name: 2024-11-29 00:00:00, dtype: float64, 'vega': Ticker
SPY    161.921427
Name: 2024-11-29 00:00:00, dtype: float64, 'rho_call': Ticker
SPY    172.159054
Name: 2024-11-29 00:00:00, dtype: float64, 'rho_put': Ticker
SPY   -121.677434
Name: 2024-11-29 00:00:00, dtype: float64}
